In [97]:
from jwst import datamodels
import pysiaf
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
import sys,os,re
import numpy as np
from astropy.modeling.models import Polynomial2D, Mapping, Shift

fgs_siaf = pysiaf.Siaf('FGS') 
fgs_aperture= fgs_siaf.apertures['FGS1_FULL']
fgs_x0=fgs_aperture.XSciRef-1.0
fgs_y0=fgs_aperture.YSciRef-1.0
dy = 0.02 # difference in pixel to go into +- y-direction
print(f'FGS V2Ref: {fgs_aperture.V2Ref}')
print(f'FGS V3Ref: {fgs_aperture.V3Ref}')
print(f'FGS V3IdlYAngle: {fgs_aperture.V3IdlYAngle}')

FGS V2Ref: 206.407
FGS V3Ref: -697.765
FGS V3IdlYAngle: -1.24120427


In [88]:
fgsfilename = '/Volumes/Ext1/jhat/v2v3ref/hawki_v1/jw01476006004_06201_00002_guider1_jhat.fits'
nrcfilename = '/Volumes/Ext1/jhat/v2v3ref/hawki_v1/jw01476006004_06101_00002_nrca1_jhat.fits'

In [89]:
fgs_model = datamodels.ImageModel(fgsfilename)
fgs_detector_to_world=fgs_model.meta.wcs.get_transform('detector', 'world') 
fgs_detector_to_v2v3=fgs_model.meta.wcs.get_transform('detector', 'v2v3') 
fgs_world_to_v2v3=fgs_model.meta.wcs.get_transform('world','v2v3') 
nrc_model = datamodels.ImageModel(nrcfilename)
nrc_detector_to_world=nrc_model.meta.wcs.get_transform('detector', 'world') 


In [90]:
fgs_v2,fgs_v3 = fgs_detector_to_v2v3(fgs_x0,fgs_y0)
print(f'V2/V3ref siaf   {fgs_aperture.V2Ref:.4f} {fgs_aperture.V3Ref:.4f}')
print(f'V2/V3ref center {fgs_v2:.4f} {fgs_v3:.4f}')

V2/V3ref siaf   206.4070 -697.7650
V2/V3ref center 206.4640 -697.9700


In [11]:
def get_RaDec12(model,x1,y1,x2,y2):
    detector_to_world=model.meta.wcs.get_transform('detector', 'world') 
    ra1,dec1 = detector_to_world(x1,y1)
    ra2,dec2 = detector_to_world(x2,y2)
    return(ra1,dec1,ra2,dec2)

def get_angles(model,ra1,dec1,ra2,dec2):
    pos1 = SkyCoord(ra=ra1, dec=dec1, unit=(u.deg,u.deg), frame='icrs')
    pos2 = SkyCoord(ra=ra2, dec=dec2, unit=(u.deg,u.deg), frame='icrs')
    PA = pos1.position_angle(pos2).to(u.deg)
    world_to_v2v3=model.meta.wcs.get_transform('world','v2v3') 
    v2_1,v3_1 = world_to_v2v3(ra1,dec1)
    v2_2,v3_2 = world_to_v2v3(ra2,dec2)
    #print(v2_1,v3_1,v2_2,v3_2)
    pos1 = SkyCoord(ra=v2_1, dec=v3_1, unit=(u.arcsec,u.arcsec), frame='icrs')
    pos2 = SkyCoord(ra=v2_2, dec=v3_2, unit=(u.arcsec,u.arcsec), frame='icrs')
    angle = pos1.position_angle(pos2).to(u.deg)
    if angle.degree>90: 
        angle-=180.0*u.deg
    if angle.degree<-90:
        angle+=180.0*u.deg
    return(PA,angle)

In [98]:
y1 = fgs_y0+dy
y2 = fgs_y0-dy
x1=x2=fgs_x0
fgs_radec12 = get_RaDec12(fgs_model,x1,y1,x2,y2)
fgs_PA,fgs_V3IdlYAngle = get_angles(fgs_model,fgs_radec12[0],fgs_radec12[1],fgs_radec12[2],fgs_radec12[3])
print(f'fgs_V3IdlYAngle: {fgs_V3IdlYAngle.degree} (nominal fgs_V3IdlYAngle:{fgs_aperture.V3IdlYAngle})')


fgs_V3IdlYAngle: -1.2508098372933034 (nominal fgs_V3IdlYAngle:-1.24120427)


In [99]:
fgs_v2_1,fgs_v3_1 = fgs_detector_to_v2v3(x1,y1)
fgs_v2_2,fgs_v3_2 = fgs_detector_to_v2v3(x2,y2)
pos1 = SkyCoord(ra=fgs_v2_1, dec=fgs_v3_1, unit=(u.arcsec,u.arcsec), frame='icrs')
pos2 = SkyCoord(ra=fgs_v2_2, dec=fgs_v3_2, unit=(u.arcsec,u.arcsec), frame='icrs')
angle = pos1.position_angle(pos2).to(u.deg)
if angle.degree>90: 
    angle-=180.0*u.deg
if angle.degree<-90:
    angle+=180.0*u.deg
print(angle.deg)

-1.2508099712959222


In [100]:
angle = np.degrees(np.arctan2(fgs_v2_2-fgs_v2_1, fgs_v3_2-fgs_v3_1))
print(angle-180)
angle = np.degrees(np.arctan2(fgs_v2_2-fgs_v2, fgs_v3_2-fgs_v3))
print(angle-180)
angle = np.degrees(np.arctan2(fgs_v2-fgs_v2_1, fgs_v3-fgs_v3_1))
print(angle-180)

-1.2508171302552
-1.2508199558254205
-1.250814304683388


In [26]:
def rotate_v2v3(x0,y0,angle,par=1.0):
    # Cast the transform functions as 1st order polynomials
    xc = {}
    yc = {}
    xc['c1_0'] = par * np.cos(angle)
    xc['c0_1'] = par * (0. - np.sin(angle))
    yc['c1_0'] = np.sin(angle)
    yc['c0_1'] = np.cos(angle)

    # center
    xc['c0_0'] = x0
    yc['c0_0'] = y0

    xmodel = Polynomial2D(1, **xc)
    ymodel = Polynomial2D(1, **yc)

    return xmodel, ymodel


In [48]:
v2function, v3function = rotate_v2v3(fgs_v2,fgs_v3,5.0/180*np.pi)
v2function, v3function = rotate_v2v3(0.,0.,10.0 * np.pi / 180.)
v2list = np.array([fgs_v2,fgs_v2_1,fgs_v2_2]) - fgs_v2
v3list = np.array([fgs_v3,fgs_v3_1,fgs_v3_2]) - fgs_v3
v2new = v2function(v2list,v3list) + fgs_v2
v3new = v2function(v2list,v3list) + fgs_v3
print(v2new,v3new)

[206.464      206.45035656 206.47764417] [-697.97       -697.98364344 -697.95635583]


In [49]:
pos1 = SkyCoord(ra=v2new[1], dec=v3new[1], unit=(u.arcsec,u.arcsec), frame='icrs')
pos2 = SkyCoord(ra=v2new[2], dec=v3new[2], unit=(u.arcsec,u.arcsec), frame='icrs')
angle = pos1.position_angle(pos2).to(u.deg)
if angle.degree>90: 
    angle-=180.0*u.deg
if angle.degree<-90:
    angle+=180.0*u.deg
print(angle.deg)

44.99983599658119


In [76]:
v2function, v3function = rotate_v2v3(fgs_v2,fgs_v3,5.0/180*np.pi)
v2function, v3function = rotate_v2v3(0.,0.,10.0 * np.pi / 180.)
v2list = np.array([0,0,0])
v3list = np.array([-1,0,1])
v2new = v2function(v2list,v3list)
v3new = v3function(v2list,v3list)
print(v2new,v3new)

[ 0.17364818  0.         -0.17364818] [-0.98480775  0.          0.98480775]


In [75]:
from astropy.modeling.rotations import Rotation2D
R = Rotation2D()
print(R)
(v2new,v3new) = R.evaluate(v2list,v3list,10*u.deg)
print(v2list,v3list)
print(v2new,v3new)

Model: Rotation2D
Inputs: ('x', 'y')
Outputs: ('x', 'y')
Model set size: 1
Parameters:
    angle
    -----
      0.0
[0 0 0] [-1  0  1]
[ 0.17364818  0.         -0.17364818] [-0.98480775  0.          0.98480775]


In [80]:
angle = np.degrees(np.arctan2(v2new[2]-v2new[1], v3new[2]-v3new[1]))
print(angle)

-10.0
